## Getting Started

### Importing

In [17]:
import tensorflow as tf

### Computational Graph

In [18]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)

print(node1,node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


In [19]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


So, we can add the third node which add node1 and node2.
<img src="https://www.tensorflow.org/images/getting_started_add.png" />
As it stands, this graph is not especially interesting because it always produces a constant result.

In [20]:
node3 = tf.add(node1,node2)
print("Node3:",node3)
print("Session.run(node3)", sess.run(node3))

Node3: Tensor("Add_1:0", shape=(), dtype=float32)
Session.run(node3) 7.0


- Placeholder: Used for parameterized to accept externam inputs. A placeholder is a promise to provide a value later
<img src="https://www.tensorflow.org/images/getting_started_adder.png" />

In [21]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # provides a shortcut to tf.add(a,b)

In [22]:
print(sess.run(adder_node,{a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


We can also make complex computation graph by using multiplication operation. The preceding computational graph would look as follows in TensorBoard:

<img src="https://www.tensorflow.org/images/getting_started_triple.png">


In [23]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple,{a: 3, b: 4.5}))

22.5


To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value.

In [24]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [25]:
init = tf.global_variables_initializer()
sess.run(init)

In [26]:
print(sess.run(linear_model, {x: [1,2,3,4,5]}))

[ 0.          0.30000001  0.60000002  0.90000004  1.20000005]


## Loss Function

In [27]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)

As you can see EPE is very high. We can manually adjust the value of W and b to reduce the EPE. 

## Training API

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)
for i in range(1000):
    sess.run(train, { x:[1,2,3,4,5], y:[0,-1,-2,-3,-4] })

print(sess.run([W,b]))

[array([-0.99999952], dtype=float32), array([ 0.99999833], dtype=float32)]


In [29]:
import numpy as np
import tensorflow as tf

#Model Parameters 
W = tf.Variable([0.3], tf.float32)
b = tf.Variable([0.3], tf.float32)

#Model input and outputs 
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
    
#Model
linear_model = W * x + b

#Loss
loss = tf.reduce_sum(tf.square(linear_model - y))

#optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

#train data
x_train = [1,3,2,4]
y_train = [0,-2,-1,-3]

#initialization
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# training loop
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
    
#evaluate training accuracy
curr_W, curr_b, curr_loss =  sess.run([W,b, loss], {x: x_train, y: y_train})
print("W: %s b: %s Loss: %s" %(curr_W, curr_b, curr_loss))


W: [-0.99999791] b: [ 0.99999392] Loss: 2.52847e-11


## Tf.contrib.learn

In [2]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,num_epochs=1000)
#input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
print(estimator.evaluate(input_fn=input_fn))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa86cf49a90>, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_task_id': 0, '_evaluation_master': '', '_save_summary_steps': 100, '_environment': 'local', '_task_type': None, '_tf_random_seed': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpg_ev53iy/mod

### A custom model